In [1]:
!pip install icecream

In [2]:
# importing the libraries
import pandas as pd
import numpy as np
from tqdm import tqdm
from icecream import ic

# for reading and displaying images
from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline

# for creating validation set
from sklearn.model_selection import train_test_split

# for evaluating the model
from sklearn.metrics import accuracy_score

# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
import torchvision
import torch.nn.functional as F

In [3]:
%time
!git clone https://github.com/haroldlkh/school_image_datasets.git

CPU times: user 1 µs, sys: 1e+03 ns, total: 2 µs
Wall time: 4.77 µs
Cloning into 'school_image_datasets'...
remote: Enumerating objects: 5859, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 5859 (delta 0), reused 3 (delta 0), pack-reused 5856
Receiving objects: 100% (5859/5859), 1.46 GiB | 38.16 MiB/s, done.
Checking out files: 100% (5849/5849), done.


In [4]:
img_size=128
resize = torchvision.transforms.Resize((img_size, img_size))
to_tensor = torchvision.transforms.ToTensor()
transform = torchvision.transforms.Compose([resize, to_tensor])

ds = torchvision.datasets.ImageFolder("/content/school_image_datasets/data607/plant_diseases", transform=transform)

In [5]:
n=len(ds)
n_tr=int(n*.8)
n_te=n-n_tr
# n_val=int(n_te*.5)
# n_test=n_te-n_val
ds_train, ds_te = torch.utils.data.random_split(ds, [n_tr, n_te])
# ds_val, ds_test = torch.utils.data.random_split(ds_te, [n_val, n_test])

In [6]:
dl_train = torch.utils.data.DataLoader(ds_train, batch_size=32, shuffle=True)
dl_val = torch.utils.data.DataLoader(ds_te, batch_size=len(ds_te))
# dl_test = torch.utils.data.DataLoader(ds_test, batch_size=len(ds_test))
x_val, y_val = next(iter(dl_val))
x_val = x_val.to(0)
y_val = y_val.to(0)

In [7]:
dl_ = torch.utils.data.DataLoader(ds_train, batch_size=len(ds_train), shuffle=True)
x_, y_ = next(iter(dl_))
x_ = x_.to(0)
y_ = y_.to(0)

In [8]:
mean = torch.mean(x_, dim=[0, 2, 3])
std = torch.std(x_, dim=[0, 2, 3])
print(mean, std)

tensor([0.4703, 0.5885, 0.3978], device='cuda:0') tensor([0.1839, 0.1672, 0.1921], device='cuda:0')


In [9]:
random_horizontal_flip = torchvision.transforms.RandomHorizontalFlip()
random_vertical_flip = torchvision.transforms.RandomVerticalFlip()
normalise = torchvision.transforms.Normalize(mean, std)

class Model(torch.nn.Module):
  def __init__(self, feature_extractor, n_features):
    super().__init__()
    self.L = torch.nn.Linear(n_features, 3)
    self.fe=feature_extractor
    self.fe.requires_grad_(False)

  def forward(self, x):
    x = normalise(x)
    if self.training:
      x = random_horizontal_flip(x)
      x = random_vertical_flip(x)
    x = self.fe(x).squeeze()
    x = self.L(x).squeeze()
    return x

def trainLeaves(dl_train, epochs=10, lr=.001):
  optimizer = torch.optim.Adam(model.parameters(), lr=lr)
  for i in tqdm(range(epochs)):
    model.train()
    train_losses = []
    for x, y in dl_train:
      y_pred = model(x.to(0))
      train_loss = F.cross_entropy(y_pred, y.to(0))
      train_losses.append(train_loss.item())
      train_loss.backward()
      optimizer.step()
      optimizer.zero_grad()
    model.eval()
    with torch.no_grad():
      y_pred = model(x_val)
      val_loss = F.cross_entropy(y_pred, y_val)
      val_acc = torch.mean(1.0*(torch.argmax(y_pred, axis=1) == y_val))
      print(f"Epoch {i+1}: mean training loss = {np.mean(train_losses):.4f}, test loss = {val_loss.item():.4f}, test accuracy = {val_acc.item():.4f}")

In [10]:
resnet = torchvision.models.resnet50(pretrained=True).to(0)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [11]:
feature_extractor = torch.nn.Sequential(*[child for child in resnet.children()][:-1])
model = Model(feature_extractor, 2048).to(0)

trainLeaves(dl_train)

 10%|█         | 1/10 [03:44<33:38, 224.29s/it]

Epoch 1: mean training loss = 0.7033, test loss = 0.3651, test accuracy = 0.9121


 20%|██        | 2/10 [07:27<29:50, 223.84s/it]

Epoch 2: mean training loss = 0.3387, test loss = 0.2892, test accuracy = 0.9088


 30%|███       | 3/10 [11:08<25:57, 222.44s/it]

...

Epoch 8: mean training loss = 0.1933, test loss = 0.1615, test accuracy = 0.9577


 90%|█████████ | 9/10 [33:22<03:42, 222.92s/it]

Epoch 9: mean training loss = 0.1531, test loss = 0.1426, test accuracy = 0.9544


100%|██████████| 10/10 [37:04<00:00, 222.43s/it]

Epoch 10: mean training loss = 0.1696, test loss = 0.1443, test accuracy = 0.9544
